In [1]:
# 🚀 Install required libraries
!pip install -q transformers datasets evaluate nltk rouge_score



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires

In [2]:
import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import nltk

# ✅ Download NLTK tokenizer
nltk.download('punkt')

# ✅ Enable GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ Using device: cuda


In [3]:
from datasets import load_dataset
from collections import Counter

# ✅ Load LEDGAR Dataset
ledgar_dataset = load_dataset("lex_glue", "ledgar")

# ✅ Extract Label Mapping (ID → Clause Category)
LABEL_MAPPING = {idx: label for idx, label in enumerate(ledgar_dataset["train"].features["label"].names)}

# ✅ Print Label Mapping
print("\n📌 LEDGAR Label Mapping (ID → Clause Category):")
print(LABEL_MAPPING)

# ✅ Check Dataset Balance
label_counts = Counter(ledgar_dataset["train"]["label"])
print("\n📊 LEDGAR Dataset Label Distribution:")
for label, count in label_counts.items():
    print(f"{LABEL_MAPPING[label]}: {count} samples")


README.md:   0%|          | 0.00/34.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/20.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.31M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.44M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]


📌 LEDGAR Label Mapping (ID → Clause Category):
{0: 'Adjustments', 1: 'Agreements', 2: 'Amendments', 3: 'Anti-Corruption Laws', 4: 'Applicable Laws', 5: 'Approvals', 6: 'Arbitration', 7: 'Assignments', 8: 'Assigns', 9: 'Authority', 10: 'Authorizations', 11: 'Base Salary', 12: 'Benefits', 13: 'Binding Effects', 14: 'Books', 15: 'Brokers', 16: 'Capitalization', 17: 'Change In Control', 18: 'Closings', 19: 'Compliance With Laws', 20: 'Confidentiality', 21: 'Consent To Jurisdiction', 22: 'Consents', 23: 'Construction', 24: 'Cooperation', 25: 'Costs', 26: 'Counterparts', 27: 'Death', 28: 'Defined Terms', 29: 'Definitions', 30: 'Disability', 31: 'Disclosures', 32: 'Duties', 33: 'Effective Dates', 34: 'Effectiveness', 35: 'Employment', 36: 'Enforceability', 37: 'Enforcements', 38: 'Entire Agreements', 39: 'Erisa', 40: 'Existence', 41: 'Expenses', 42: 'Fees', 43: 'Financial Statements', 44: 'Forfeitures', 45: 'Further Assurances', 46: 'General', 47: 'Governing Laws', 48: 'Headings', 49: 'Indem

In [4]:
# ✅ Extract Label Mapping (ID → Clause Category)
labels_ids2names = {idx: label_name for idx, label_name in enumerate(ledgar_dataset["train"].features["label"].names)}

print("\n📌 LEDGAR Label Mapping (ID → Clause Category):")
print(labels_ids2names)



📌 LEDGAR Label Mapping (ID → Clause Category):
{0: 'Adjustments', 1: 'Agreements', 2: 'Amendments', 3: 'Anti-Corruption Laws', 4: 'Applicable Laws', 5: 'Approvals', 6: 'Arbitration', 7: 'Assignments', 8: 'Assigns', 9: 'Authority', 10: 'Authorizations', 11: 'Base Salary', 12: 'Benefits', 13: 'Binding Effects', 14: 'Books', 15: 'Brokers', 16: 'Capitalization', 17: 'Change In Control', 18: 'Closings', 19: 'Compliance With Laws', 20: 'Confidentiality', 21: 'Consent To Jurisdiction', 22: 'Consents', 23: 'Construction', 24: 'Cooperation', 25: 'Costs', 26: 'Counterparts', 27: 'Death', 28: 'Defined Terms', 29: 'Definitions', 30: 'Disability', 31: 'Disclosures', 32: 'Duties', 33: 'Effective Dates', 34: 'Effectiveness', 35: 'Employment', 36: 'Enforceability', 37: 'Enforcements', 38: 'Entire Agreements', 39: 'Erisa', 40: 'Existence', 41: 'Expenses', 42: 'Fees', 43: 'Financial Statements', 44: 'Forfeitures', 45: 'Further Assurances', 46: 'General', 47: 'Governing Laws', 48: 'Headings', 49: 'Indem

In [5]:
# ✅ Mapping LEDGAR Labels to Our Project Categories
LEGAL_CLAUSE_CATEGORIES = {
    "Obligations": ["Duties", "Compliance With Laws", "Responsibilities", "Liabilities"],
    "Termination Clauses": ["Terminations", "Survival"],
    "Confidentiality Agreements": ["Confidentiality", "Non-Disparagement"],
    "Payment Terms": ["Payments", "Fees", "Tax Withholdings", "Taxes"],
    "Governing Law Provisions": ["Governing Laws", "Jurisdictions", "Submission To Jurisdiction"]
}

# ✅ Reverse Mapping for Easy Lookup
LEDGAR_TO_PROJECT_CATEGORIES = {
    label: category for category, labels in LEGAL_CLAUSE_CATEGORIES.items() for label in labels
}

print("\n✅ LEDGAR Labels Mapped to Our Categories:")
print(LEDGAR_TO_PROJECT_CATEGORIES)



✅ LEDGAR Labels Mapped to Our Categories:
{'Duties': 'Obligations', 'Compliance With Laws': 'Obligations', 'Responsibilities': 'Obligations', 'Liabilities': 'Obligations', 'Terminations': 'Termination Clauses', 'Survival': 'Termination Clauses', 'Confidentiality': 'Confidentiality Agreements', 'Non-Disparagement': 'Confidentiality Agreements', 'Payments': 'Payment Terms', 'Fees': 'Payment Terms', 'Tax Withholdings': 'Payment Terms', 'Taxes': 'Payment Terms', 'Governing Laws': 'Governing Law Provisions', 'Jurisdictions': 'Governing Law Provisions', 'Submission To Jurisdiction': 'Governing Law Provisions'}


In [6]:
from transformers import AutoTokenizer
import numpy as np

# ✅ Load Tokenizer
MODEL_NAME = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_data(dataset):
    """Tokenizes LEDGAR dataset and ensures labels are correctly formatted for PyTorch/TensorFlow."""

    def tokenize_function(examples):
        encoding = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

        # ✅ Ensure labels are stored as a NumPy array (Fix TypeError)
        encoding["label"] = np.array(examples["label"], dtype=np.int64)  # Convert list of labels into NumPy array

        return encoding

    return dataset.map(tokenize_function, batched=True, remove_columns=["text"], num_proc=1)  # ✅ Ensure correct formatting

# ✅ Tokenize LEDGAR dataset
tokenized_ledgar = preprocess_data(ledgar_dataset)

# ✅ Create train-validation split
train_test_split = tokenized_ledgar["train"].train_test_split(test_size=0.1)
tokenized_ledgar["train"], tokenized_ledgar["validation"] = train_test_split["train"], train_test_split["test"]

print("✅ Tokenization complete! Train size:", len(tokenized_ledgar["train"]), "Validation size:", len(tokenized_ledgar["validation"]))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

✅ Tokenization complete! Train size: 54000 Validation size: 6000


In [7]:
import torch
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# ✅ Load Pretrained Model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_MAPPING)).to("cuda")

# ✅ Define Training Arguments
training_args = TrainingArguments(
    output_dir="./models/bert-ledgar",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs/bert-ledgar",
    logging_steps=50,
    save_total_limit=2,
    fp16=True,  # ✅ Enable Mixed Precision for Speed
)

# ✅ Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ledgar["train"],
    eval_dataset=tokenized_ledgar["validation"]
)

# ✅ Start Training
trainer.train()

# ✅ Save Fine-Tuned Model
model.save_pretrained("./models/bert-ledgar")
tokenizer.save_pretrained("./models/bert-ledgar")

print("✅ Fine-Tuning Complete for BERT on LEDGAR!")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kanishk6101 (kanishk6101-purdue-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.657700,0.659142
2,0.284200,0.530886


✅ Fine-Tuning Complete for BERT on LEDGAR!


In [45]:
import numpy as np
import evaluate
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# ✅ Load Fine-Tuned Model
model_path = "./models/bert-ledgar"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to("cuda")

# ✅ Get Evaluation Dataset
eval_dataset = tokenized_ledgar["validation"]

# ✅ Define Trainer for Evaluation
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset
)

# ✅ Run Evaluation
results = trainer.evaluate()
print("\n📊 Evaluation Results:", results)

# ✅ Run Predictions on Validation Set
predictions = trainer.predict(eval_dataset).predictions
predicted_labels = np.argmax(predictions, axis=1)

# ✅ Extract True Labels
true_labels = np.array(eval_dataset["label"])

# ✅ Compute Evaluation Metrics
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

classification_metrics = {
    "accuracy": round(accuracy.compute(predictions=predicted_labels, references=true_labels)["accuracy"] * 100, 2),
    "precision": round(precision.compute(predictions=predicted_labels, references=true_labels, average="macro")["precision"] * 100, 2),
    "recall": round(recall.compute(predictions=predicted_labels, references=true_labels, average="macro")["recall"] * 100, 2),
    "f1": round(f1.compute(predictions=predicted_labels, references=true_labels, average="macro")["f1"] * 100, 2)
}

print("\n📊 Updated Classification Metrics (Percentage):")
print(f"✅ Accuracy: {classification_metrics['accuracy']}%")
print(f"✅ Precision: {classification_metrics['precision']}%")
print(f"✅ Recall: {classification_metrics['recall']}%")
print(f"✅ F1 Score: {classification_metrics['f1']}%")



📊 Evaluation Results: {'eval_loss': 0.511886715888977, 'eval_model_preparation_time': 0.0029, 'eval_runtime': 11.7494, 'eval_samples_per_second': 510.664, 'eval_steps_per_second': 31.917}

📊 Updated Classification Metrics (Percentage):
✅ Accuracy: 86.63%
✅ Precision: 80.22%
✅ Recall: 78.06%
✅ F1 Score: 78.45%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
import os
import numpy as np
from datasets import Dataset
import evaluate

# Define the path to your labeled sample file.
sample_file_path = "/content/sample_data/master_services_agreement.txt"

# Load and parse the file.
# Each line should be formatted as: clause text<TAB>label
real_world_data = []
with open(sample_file_path, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) == 2:
            text, label = parts
            try:
                label = int(label)
                real_world_data.append({"text": text, "label": label})
            except ValueError:
                # Skip lines with invalid label format.
                continue

if not real_world_data:
    raise ValueError("No valid test data found in the file. Ensure each line is formatted as 'clause<TAB>label'.")

# Create a Hugging Face Dataset from the list.
real_world_dataset = Dataset.from_list(real_world_data)

# Define a tokenization function for the real-world dataset.
def tokenize_real_world(example):
    encoding = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
    encoding["labels"] = example["label"]
    return encoding

# Tokenize the dataset.
tokenized_real_world = real_world_dataset.map(tokenize_real_world, batched=False)
tokenized_real_world = tokenized_real_world.remove_columns(["text"])

# Run predictions using the Trainer.
predictions_output = trainer.predict(tokenized_real_world)
logits = predictions_output.predictions
predicted_labels = np.argmax(logits, axis=-1)
true_labels = np.array(tokenized_real_world["labels"])

# Load evaluation metrics.
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

# Compute metrics.
accuracy_value = accuracy_metric.compute(predictions=predicted_labels, references=true_labels)["accuracy"]
precision_value = precision_metric.compute(predictions=predicted_labels, references=true_labels, average="macro")["precision"]
recall_value = recall_metric.compute(predictions=predicted_labels, references=true_labels, average="macro")["recall"]
f1_value = f1_metric.compute(predictions=predicted_labels, references=true_labels, average="macro")["f1"]

print("\n📊 Real-World Test Metrics:")
print(f"✅ Accuracy: {accuracy_value * 100:.2f}%")
print(f"✅ Precision: {precision_value * 100:.2f}%")
print(f"✅ Recall: {recall_value * 100:.2f}%")
print(f"✅ F1 Score: {f1_value * 100:.2f}%")


Map:   0%|          | 0/73 [00:00<?, ? examples/s]


📊 Real-World Test Metrics:
✅ Accuracy: 38.36%
✅ Precision: 23.74%
✅ Recall: 16.15%
✅ F1 Score: 17.84%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
